[View in Colaboratory](https://colab.research.google.com/github/sthalles/tensorflow-tutorials/blob/master/Day_1_Pre_Made_Estimators_Solutions.ipynb)

# Pre-Made Estimators

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

The Estimator is a high-level Tensorflow API. 

![alt text](https://www.tensorflow.org/images/tensorflow_programming_environment.png)

It provides functionalities for representing a complete model.
1. Building the actual model
2. Initialization
3. Logging
3. Saving and restoring
2. Measuring
3. Testing

To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

1.  Create one or more **input functions**.
2.  Define the model's **feature columns**.
3. Instantiate an Estimator, specifying the feature columns and various hyperparameters.
4. Train and Evaluate

# Loading data

In [4]:
"""DO NOT NEED CHANGES"""
def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

In [5]:
"""DO NOT NEED CHANGES"""
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


def load_dataset():
  train_path, test_path = maybe_download()
  train_data = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
  test_data = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
  
  train_input = train_data[CSV_COLUMN_NAMES[0:-1]]
  train_labels = train_data[CSV_COLUMN_NAMES[-1]]
  
  test_input = test_data[CSV_COLUMN_NAMES[0:-1]]
  test_labels = test_data[CSV_COLUMN_NAMES[-1]]
  
  return (train_input, train_labels), (test_input, test_labels)

![alt text](https://www.tensorflow.org/images/iris_three_species.jpg)

In [6]:
(X_train, y_train), (X_test, y_test) = load_dataset()

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


The Iris data set contains 4 features and 1 label. 

The 4 **features** identify the following botanical characteristics of individual Iris flowers:

- sepal length
- sepal width
- petal length
- petal width

Our model will represent these features as float32 numerical data.

The **label** identifies the Iris species, which must be one of the following:

- Iris setosa (0)
- Iris versicolor (1)
- Iris virginica (2)

Our model will represent the label as int32 categorical data.

In [10]:
X_train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [11]:
y_train.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

# Creating Input funcions

The Estimator expects an input_function() to return a [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) or a tuple of *(features, labels)*.

Let's use the tf.data.Dataset as our input streaming.
-  [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) is the recommended Tensorflow input pipeline.

In [0]:
def train_input_fn(features, labels, batch_size):
  """
  A function that provides input data for training as minibatches
  Return: A 'tf.data.Dataset' object: tuple (features, labels). 
          Or tuple (features, labels)
  """
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  dataset = dataset.shuffle(buffer_size=100)
  dataset = dataset.repeat()
  dataset = dataset.batch(batch_size)
  return dataset

In [0]:
def eval_input_fn(features, labels, batch_size):
  
  features = dict(features)
  if labels is None:
    inputs = features
  else:
    inputs = (dict(features), labels)
  
  dataset = tf.data.Dataset.from_tensor_slices(inputs)
  dataset = dataset.repeat(1)
  dataset = dataset.batch(batch_size)
  return dataset

# Defining the Feature Columns

Now, we have to convert our data into **Tensors** so that our Model can use it.

![alt text](https://www.tensorflow.org/images/feature_columns/some_constructors.jpg)

**Feature Columns** define the **type of features** we are going to feed into our Models.

Since the 4 feature columns of the Iris dataset are represented as continuos values, we need to especify it when creating the feature columns.

To do that, we use the: [tf.feature_column.numeric_column()](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) constructor.

  - **tf.feature_column.numeric_column()** Represents real valued or numerical features.
  
Info++: [Feature Engineering](https://www.tensorflow.org/get_started/feature_columns)
  
 


In [0]:
# feature columns define how to use the feature data
my_feature_columns = []
for feature_column in X_train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=feature_column))

# Hyperparameters

1. Tune the hyperparameters bellow.

In [0]:
learning_rate = 0.1
number_of_classes = 3
batch_size = 16
max_step = 1000
regularization = 0.001


# Building the Estimator

An Estimator encapsulates all the necessary parts of a model. 

Some of the available Estimators include:

1. **BoostedTrees** Classifier/Regressor
2. **DNN Classifier**/Regressor
3. **DNNLinearCombined** Classifier/Regressor
4. **Linear** Classifier/Regressor

Checkout: [tf.estimator](https://www.tensorflow.org/api_docs/python/tf/estimator)

## Exercise

1. Use the **tf.estimator.LinearClassifier** to classify the Iris Dataset.

Things to keep in mind.
  - Linear models are very simple, for this case, pay special attention to the **number of classes** and the **learning rate** tunning.
  - Play with different configurations of **batch size**, it can have dramatic effects on how quick the model converges.

In [0]:
classifier = tf.estimator.LinearClassifier(    
    feature_columns=my_feature_columns,
    n_classes=number_of_classes,
    optimizer=tf.train.FtrlOptimizer(
      learning_rate=learning_rate,
      l1_regularization_strength=regularization
    ))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk6ty2hho', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f39aca0ef28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
classifier.train(input_fn=lambda: train_input_fn(X_train, y_train, batch_size=batch_size), 
                 steps=max_step #  Number of steps for which to train model.
                )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpk6ty2hho/model.ckpt.
INFO:tensorflow:loss = 17.577797, step = 0
INFO:tensorflow:global_step/sec: 278.412
INFO:tensorflow:loss = 6.420633, step = 100 (0.364 sec)
INFO:tensorflow:global_step/sec: 301.772
INFO:tensorflow:loss = 4.9447355, step = 200 (0.333 sec)
INFO:tensorflow:global_step/sec: 302.855
INFO:tensorflow:loss = 6.3400536, step = 300 (0.328 sec)
INFO:tensorflow:global_step/sec: 255.452
INFO:tensorflow:loss = 5.9765987, step = 400 (0.395 sec)
INFO:tensorflow:global_step/sec: 251.835
INFO:tensorflow:loss = 4.7573376, step = 500 (0.393 sec)
INFO:tensorflow:global_step/sec: 243.326
INFO:tensorflow:loss = 5.26486, step = 600 (0.415 sec)
INFO:tensorflow:global_step/sec: 252.541
INFO:tensorflow:los

In [0]:
classifier.evaluate(input_fn=lambda: eval_input_fn(X_test, y_test, batch_size=512))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-14-16:16:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk6ty2hho/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-14-16:16:07
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.93333334, average_loss = 0.26765403, global_step = 1000, loss = 8.029621


{'accuracy': 0.93333334,
 'average_loss': 0.26765403,
 'global_step': 1000,
 'loss': 8.029621}

# Deep Neural Networks

### Definition:

"*...a computing system made up of a number of simple, highly interconnected processing elements, which process information by their dynamic state response to external inputs.*"

- Function approximator
- Very powerful for non-linear relationships
- Represents a function as the composition of many functions.

## Exercise

1. Change the Linear Model Estimator to a Deep Neural Network.
- Head over to the Tensorflow documentation for [tf.estimator.DNNClassifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) and check it out.

Think about:

- How many layers you need
- The number of units in each layer
- The activation function used by default
- The Gradient Descent Optimizer. 

![alt text](https://www.tensorflow.org/images/custom_estimators/full_network.png)

## Architecturing your network

![LeNet-5](http://cs231n.github.io/assets/nn1/layer_sizes.jpeg)

Neural Nets with more hidden layers are able to represent more complex functions. 

- With more power comes complicated decision boundaries.

Take care with **Overfitting**!

- It occurs when a model with **high capacity** fits the noise in the data instead of the (assumed) underlying relationship.


# Regularization

Effects of Regularization. The figure below shows the decision boundaries of the same DNN (20 hidden units), with different regularization penalties. 

Note that more regularization smooths the decision boundary.

- It fights **Overfitting**.

![alt text](http://cs231n.github.io/assets/nn1/reg_strengths.jpeg)